In [1]:
# !pip install mwparserfromhell

In [2]:
# !pip uninstall regex -y
# !pip install regex==2022.3.2

In [3]:
#Basic Imports
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

# Text Preprocessing
import re
import xml
import json
from json2html import *

#Web Scraping
from mwparserfromhell import parse
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver_path = "chromedriver.exe"

# Define Brave path
brave_path = "C:/Program Files/BraveSoftware/Brave-Browser/Application/brave.exe"
options = webdriver.ChromeOptions()
options.binary_location = brave_path

# Create new automated instance of Brave
driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()),options = options)
# titles = ['International_Space_Station']


# 
# Beezie_Madden
# McLain_Ward
# Rodrigo_Pessoa
# Michael_Jung
# Laura_Kraut
# Nick_Skelton
# Ingrid_Klimke
# Kent_Farrington
# Ludger_Beerbaum
# Mary_King
# Mark_Todd
# William_Fox-Pitt
# Charlotte_Casiraghi
# Georgina_Bloomberg
# Edwina_Tops-Alexander
# Jessica_Springsteen
# Zara_Tindall
# 'Charlotte_Dujardin', 'Beezie_Madden', 'Jessica_von_Bredow-Werndl' 'Steffen_Peters', 'Sabine_Schut-Kery'
# 'Tiger_Woods', 'Jon_Rahm', 'Rory_McIlroy', 'Justin_Thomas', 'Scottie_Scheffler', 'Collin_Morikawa'
# 'Maxim_Mikhaylov_(volleyball)', 'Osmany_Juantorena', 'Sergey_Tetyukhin', 'Wallace_de_Souza', 'Bruno_Rezende'
# 'Michael_Phelps', 'Dara_Torres', 'Caeleb_Dressel', 'Katie_Ledecky', 'Sun_Yang'
# Viswanathan_Anand, Ding_Liren, Ian_Nepomniachtchi, Alireza_Firouzja, Wesley_So
# 'Fan_Zhendong', 'Ma_Long', 'Hugo_Calderano', 'Tomokazu_Harimoto', 'Dimitrij_Ovtcharov', 'Timo_Boll'
#'P._V._Sindhu', 'Akane_Yamaguchi', 'Chen_Yufei', 'Tai_Tzu-ying', 'An_Se-young','He_Bingjiao', 'Ratchanok_Intanon', 'Wang_Zhiyi'
# 'Busanan_Ongbamrungphan', 'Han_Yue', 'Pornpawee_Chochuwong', 'Nozomi_Okuhara', 'Michelle_Li'
# ,'Nicol_David','Natalie_Grainger','Linda_Elriani', 'Natalie_Grinham', 'Rachael_Grinham' , Ali_Farag, 'Diego_Elías', 'Tarek_Momen', 'Karim_Abdel_Gawad'
# 'James_Willstrop', 'Nick_Matthew', 'Ramy_Ashour', 'Grégory_Gaultier', 'Karim_Darwish_(squash)', 'Amr_Shabana', 'Lee_Beachill'
# 'Nouran_Gohar', 'Nour_El_Sherbini', 'Joelle_King', 'Nour_El_Tayeb', 'Sarah-Jane_Perry', 'Laura_Massaro', 'Raneem_El_Weleily'
# links = []
# for page_title in titles:
#     history_page = f"https://en.wikipedia.org/w/index.php?title={page_title}&action=history"
#     driver.get(history_page)
#     inter_links = driver.find_elements(By.PARTIAL_LINK_TEXT,", ")
#     for link in inter_links:
#         links.append(link.get_attribute('href')+'&action=raw')

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.30M/6.30M [00:00<00:00, 14.9MB/s]


In [38]:
# Functions
def remove_html_spaces(text):
    if "&nbsp;" in text:
        return re.sub("&nbsp;", ' ', text)
    else:
        return re.sub("&nbsp", ' ', text)
def replace_br(text):
    if '<br/>' in text:
        return re.sub('\<br\/\>', ', ', text)
    elif '<br>' in text:
        return re.sub('\<br\>', ', ', text)
    return text
def replace_newlines(text):
    return re.sub("\n", '<>', text)
def replace_newlines_tabs(text):
    return re.sub("\n\|", '\n||', text)
def replace_tabs(text):
    return re.sub(" \| ", 'ZXC^', text)
def replace_tabs_alt(text):
    return re.sub("\|", 'ZXC^', text)
def replace_stars(text):
    return re.sub("\*", 'ZXC^', text)
def replace_special(text):
    return re.sub("<>", ' ', text)
def remove_comments(text):
    return re.sub('<!--.*?-->','',text)
def remove_braces(text):
    text = re.sub('{{','',text)
    return re.sub('}}','',text)
def remove_html(text):
    return re.sub('(?<=\<)(.*?)(?=\>)','',text)
def remove_refs(text):
    return re.sub('(?<=\<ref\>)(.*?)(?=\<\/ref\>)','',text)
def remove_pixel_info(text):
    return re.sub(r'\d+px', '', text)
def extract_text_between_braces(string):
    return re.findall(r'\[\[([^\[\]]+)\]\]', string)
def extract_text_between_curly_braces(string):
    return re.findall(r'{{(.*?)}}', string)

def clean(text):
    # remove html tags
    result = remove_refs(text)
    result = replace_br(result)
    result = remove_pixel_info(result)
    result = remove_html_spaces(result)
    result = remove_html(result) 
    result = replace_special(result)
    lst = extract_text_between_braces(result)
    lst_alt = []
    for ele in lst:
        if '|' in ele:
            result = result.replace(ele, ele.split('|')[1])
    lst = extract_text_between_curly_braces(result)
    lst_alt = []
    for ele in lst:
        if ('|' in ele) and ('hlist' not in ele) and ('birth' not in ele):
            result = result.replace(ele, ele.split('|')[1])
    result = re.sub('[\[\]]+','',result) # remove [[around words]]
    return result

def post_clean(text):
    return remove_braces(tab_sep_alt(text.strip()))
#     return remove_braces(text.strip())

def clean_micro(text):
    text = text.strip()
    if(re.search("(?<={{)(.*?)(?=\|)",text)):
        text = text[re.search("(?<={{)(.*?)(?=\|)",text).end()+1:-2]
    return text

def extract_curly(text,phrase):
    text2 = text[re.search(phrase,text).start()+2:]
    i=2
    temp = ''
    for char in text2:
        if (i==0):
            break
        temp += char
        if char=='{':
            i = i+1
        elif char=='}':
            i = i-1
    return re.sub("{{"+temp,'',text)

def tab_sep_alt(text):
    if(re.search('\S\|\S',text)):
        text = text[re.search('\S\|\S',text).start()+2:]
    return text

def clean_square(text):
    """ Removes unwanted characters from the text """
    return text.strip().strip("[]")

def modify(json_dict,key):
    # removing curly brace part for specific cases
    try:
        temps = ["{{ctitle","{{cite","{{Cite"]
        for temp in temps:
            while(re.search(temp,json_dict[key])):
                json_dict[key] = extract_curly(json_dict[key],temp)
    except:
        pass
    try:
        if(re.search("{{flatlist",json_dict[key]) or re.search("{{flat list",json_dict[key]) or re.search("{{plainlist",json_dict[key]) or ('*' in json_dict[key])):
            if 'top score' not in key:
                json_dict[key] = replace_stars(json_dict[key]+'*')
                json_dict[key] = [remove_braces(s.strip()) for s in re.findall('\^(.*?)ZXC',json_dict[key])]
    except:
        pass
    try:
        if(re.search("{{unbulleted",json_dict[key])): #not clear yet
            try:
                json_dict[key] = [re.sub('}}','',ele) for ele in json_dict[key].split("|{{")[1:]]
            except:
                json_dict[key] = json_dict[key].split("|")[1:]
    except:
        pass
    try:
        if(re.search("{{hlist",json_dict[key])):
            json_dict[key] = replace_tabs_alt(json_dict[key]+'|')
            json_dict[key] =  [remove_braces(s.strip()) for s in re.findall('\^(.*?)ZXC',json_dict[key])]
    except:
        pass
    try:
        if(re.search("(?<={{)(.*?)(?=\|)",json_dict[key])):
            json_dict[key] = json_dict[key][:re.search("(?<={{)(.*?)(?=\|)",json_dict[key]).start()-2]+json_dict[key][re.search("(?<={{)(.*?)(?=\|)",json_dict[key]).end()+1:-2]
    except:
        pass

Write Multiple Script Variations (Pruned vs Unpruned and Full key vs Interesting Keys and Clean vs general)

In [45]:
xpath = '/html/body/pre'
# for link in links:
link = links[0]
driver.get(link)
json_dict = dict()
infobox = remove_comments(driver.find_element(By.XPATH,xpath).text) # No real assumptions here
# for word in space_words:
#     if (len(infobox.split(word)) > 1):
#         infobox = infobox.replace(word, ' ' + word)
temp = replace_special(re.findall('(?<=\{\{Infobox)(.*)',replace_newlines(infobox))[0]) #Ok
# Figuring out where }} ends to mark end of infobox
i=2
infobox2 = ''
for char in temp:
    if (i==0):
        break
    infobox2 = infobox2 + char
    if char=='{':
        i = i+1
    elif char=='}':
        i = i-1
raw_dict = dict()
raw_dict_inter = dict()
json_dict = dict()
column_list = []
global_lst = []
championship_years = {}
birth_date = ''
letter = ''
try:
    infobox2 = infobox2.split('medaltemplates')[0] + 'medaltemplates' + infobox2.split('medaltemplates')[1].replace(' | ','|')
except:
    pass
try:
    infobox2 = infobox2.split('medal_templates')[0] + 'medal_templates' + infobox2.split('medal_templates')[1].replace(' | ','|')
except:
    pass
for key_value in re.findall('\^(.*?)ZXC',replace_tabs(infobox2+' | ')):
    try:    
        key = key_value.split('=')[0]
        value = '='.join(key_value.split('=')[1:])
        if value.strip() == '':
            key = key_value.split(': ')[0]
            value = ': '.join(key_value.split(':')[1:])
        if value.strip() == '':
            key = key_value.split('=')[0]
            value = '='.join(key_value.split('=')[1:])
        # Do i need these strips??
        key = key.strip()
        raw_dict[key] = value
        raw_dict_inter[key] = clean(value)
        if ("medal" in key) and ("show" not in key):
#             value = infobox2.split('medaltemplates =')[1]
            wikicode = parse(value)
            medal_templates = wikicode.filter_templates()
            for template in medal_templates:
                try:
                    if template.name.matches("MedalSport"):
                        global_lst.append("Sport: "+clean(template.get(1).value.strip()))
                    elif template.name.matches("MedalCountry"):
                        global_lst.append("Country: "+clean(template.get(1).value.strip()))
                    elif template.name.matches("MedalCount"):
                        global_lst.append("Competition: "+clean(template.get(1).value.strip()))
                        global_lst.append("Gold Medal: "+clean(template.get(2).value.strip()))
                        global_lst.append("Silver Medal: "+clean(template.get(3).value.strip()))
                        global_lst.append("Bronze Medal: "+clean(template.get(4).value.strip()))
                        global_lst.append("Competition: "+clean(template.get(5).value.strip()))
                        global_lst.append("Gold Medal: "+clean(template.get(6).value.strip()))
                        global_lst.append("Silver Medal: "+clean(template.get(7).value.strip()))
                        global_lst.append("Bronze Medal: "+clean(template.get(8).value.strip()))
                        global_lst.append("Competition: "+clean(template.get(9).value.strip()))
                        global_lst.append("Gold Medal: "+clean(template.get(10).value.strip()))
                        global_lst.append("Silver Medal: "+clean(template.get(11).value.strip()))
                        global_lst.append("Bronze Medal: "+clean(template.get(12).value.strip()))
                    elif template.name.matches("MedalCompetition"):
                        competition = clean(template.get(1).value.strip())
                        global_lst.append("Competition: "+competition)
                    elif template.name.matches("MedalGold"):
                        try:
                            competition = clean(template.get(1).value.strip())
                        except:
                            pass
                        try:
                            year_event = clean(template.get(2).value.strip())
                        except:
                            pass
                        try:
                            event = clean(template.get(3).value.strip())
                        except:
                            pass
                        string = 'Gold Medal: '
                        try:
                            string = string + competition
                        except:
                            pass
                        try:
                            string = string + ' ' + year_event
                        except:
                            pass
                        try:
                            string = string + ' ' + event
                        except:
                            pass
                        global_lst.append(string)
                    elif template.name.matches("MedalSilver"):
                        try:
                            competition = clean(template.get(1).value.strip())
                        except:
                            pass
                        try:
                            year_event = clean(template.get(2).value.strip())
                        except:
                            pass
                        try:
                            event = clean(template.get(3).value.strip())
                        except:
                            pass
                        string = 'Silver Medal: '
                        try:
                            string = string + competition
                        except:
                            pass
                        try:
                            string = string + ' ' + year_event
                        except:
                            pass
                        try:
                            string = string + ' ' + event
                        except:
                            pass
                        global_lst.append(string)
                    elif template.name.matches("MedalBronze"):
                        try:
                            competition = clean(template.get(1).value.strip())
                        except:
                            pass
                        try:
                            year_event = clean(template.get(2).value.strip())
                        except:
                            pass
                        try:
                            event = clean(template.get(3).value.strip())
                        except:
                            pass
                        string = 'Bronze Medal: '
                        try:
                            string = string + competition
                        except:
                            pass
                        try:
                            string = string + ' ' + year_event
                        except:
                            pass
                        try:
                            string = string + ' ' + event
                        except:
                            pass
                        global_lst.append(string)
                except:
                    pass
            break
        elif key == 'WorldOpenresult':
            match = re.search("^[a-zA-Z]", value)
            if match:
                letter = match
            parsed_code = parse(value)
            links = parsed_code.filter_wikilinks()
            for link in links:
                year = link.text.strip()
                championship_years[year] = link.title.strip()
                
        elif key == 'birth_date':
            parsed_code = parse(value)
            date_template = parsed_code.filter_templates()[0]
            try:
                birth_date = birth_date + date_template.get(1).value.strip() + '-' 
            except:
                pass
            try:
                birth_date = birth_date + date_template.get(2).value.strip() + '-' 
            except:
                pass
            try:
                birth_date = birth_date + date_template.get(3).value.strip() + '-'
            except:
                pass
            try:
                birth_date = birth_date + date_template.get(4).value.strip()
            except:
                pass
        else:
            json_dict[key] = clean(value)
            
        
#         # Birth Date and Age
#         if key == 'birth_date' or key == 'death_date':
#             json_dict[key] = json_dict[key][:re.search("(?<={{)(.*?)(?=\|)",json_dict[key]).start()-2] 
#             + json_dict[key][re.search("(?<={{)(.*?)(?=\|)",json_dict[key]).end()+1:-2]
#             if 'df=yes' in json_dict[key]:
#                 json_dict[key] = re.sub('\|df=yes','',json_dict[key]).strip()
#             if 'df=y' in text:
#                 json_dict[key] = re.sub('\|df=y','',json_dict[key]).strip()

#         # Birth Place
#         if key == 'birth_place':
#             if 'full_name' in json_dict[key]:
#                 json_dict[key] = re.sub('\|full_name =','',json_dict[key]).strip()
                
#         # Bowling
#         if key == 'bowling':
#             json_dict[key] = json_dict[key].strip()
#             json_dict[key] = json_dict[key][:json_dict[key].rindex(' '):] + ' ' + tab_sep_alt(json_dict[key])
        
        # Nickname and height
        if key in ['nickname', 'height', 'birth_place', 'weight', 'trainer', 'names', 'billed',
                  'debut', 'retired', 'campus_size', 'students', 'endowment', 'vice_chancellor',
                   'religious_affiliation', 'academic_affiliations', 'president']:
            json_dict[key] = re.sub('url(.*)', '', json_dict[key])
            json_dict[key] = re.sub('https(.*)', '', json_dict[key])
            json_dict[key] = re.sub('title(.*)', '', json_dict[key])
        
        # Name and fullname
        if key in ['name', 'full_name']:
            json_dict[key] = re.sub("post-nominals","",json_dict[key])
        
        if key in ['strokes']:
            json_dict[key] = re.sub('date(.*)', '', json_dict[key])
        
#         # Top Score
#         if 'top score' in key and '*' in json_dict[key] :
#             if (json_dict[key][:-9].strip() != ''):
#                 json_dict[key] = json_dict[key][:-9].strip()
            
#         #Website
#         if key == 'website':
#             json_dict[key] = re.sub('(?<=\}\}\|).*','', json_dict[key])
        
        raw_dict_inter_inter = json_dict.copy()
        json_dict[key] = re.sub('url(.*)', '', json_dict[key])
        modify(json_dict, key)
                
        
    except:
        pass
    
# Post Modification Cleaning
for key in json_dict:
    if type(json_dict[key]) is str:
        json_dict[key] = post_clean(json_dict[key])
    elif type(json_dict[key]) is list:
        json_dict[key] = [post_clean(a) for a in json_dict[key]]
        
        
# Removing empty slots
remove = []
for key in json_dict.keys():
    try:
        if(json_dict[key].strip() == ''):
            remove.append(key)
    except:
        pass
for key in remove:
    json_dict.pop(key)

                
squash_redundant = ['image', 'alt', 'caption', 'show-medals', 'module', 'updated', 'url', 'work', 
            'child', 'image_size', 'pb', 'signature', 'signature_alt', 'module2', 'reach_footnote', 'weight_footnote',
            'height_footnote', 'footnotes', 'show-medals', "logo" 
"logo_size" ,
"logo_upright", 
"logo_alt" ,
"embedded",
"pushpin_map", 
"pushpin_label_position" ,
"map_size" ,
"pushpin_map_caption" ,
"footnotes" ,
"image" ,
"image_upright", 
"image_alt", 
"caption" ,
"native_name_lang",
"head_label",
"logo" ,
"logo_size" ,
"logo_upright", 
"logo_alt" ,
"free_label",
"free_label2",
"free",
"free2",
"sports_free_label",
"sports_free_label2",
"sports_free_label3",
"sports_free",
"sports_free2",
"sports_free3",
"station_image", "station_image_alt", "configuration_image", "configuration_image_alt", "configuration_alt", "caption",
"configuration_caption", "insignia", "insignia_caption", "station_image_caption"]

for key in squash_redundant:
    try:
        json_dict.pop(key)
    except:
        pass
    
# prev_key = ''
# for key in sorted(json_dict.keys()):
#     try:
#         if((key[:-1]==prev_key[:-1]) and (prev_key[-1] in ['','1','2','3','4','5','6','7','8''9'])):
#             if ((type(json_dict[prev_key]) is str) and (type(json_dict[key]) is str)):
#                 json_dict[prev_key] = {column_list[int(prev_key[-1].strip())-1]:json_dict[prev_key],
#                                        column_list[int(key[-1].strip())-1]:json_dict[key]}
#             elif ((type(json_dict[prev_key]) is dict) and (type(json_dict[key]) is str)):
#                 json_dict[prev_key][column_list[int(key[-1].strip())-1]] = json_dict[key]
#             elif ((type(json_dict[prev_key]) is str) and (type(json_dict[key]) is dict)):
#                 {column_list[int(prev_key[-1].strip())-1]:json_dict[prev_key]}[key[-1]] = json_dict[key]
#             elif ((type(json_dict[prev_key]) is dict) and (type(json_dict[key]) is dict)):
#                 json_dict[prev_key].extend(json_dict[key])
#             json_dict.pop(key)
#         else:
#             prev_key = key
#     except:
#         pass

# draftyear           = 
# | draftround          = 
# | draftpick           = 
# | suppdraftyear       = 
# | suppdraftround      = 
# | cfldraftyear        = 
# | cfldraftround       = 
# | cfldraftpick        = 
# | afldraftyear        = 
# | afldraftround       = 
# | afldraftpick        = 
# | undraftedyear       = 
# | expansiondraftyear  = 
# | expansiondraftround = 
# | expansiondraftpick  = 

#nfldraft
# nfl_lst = []
# for key in json_dict.keys():
#     if key in ['draftyear', 'draftround', 'draftpick']:
#         nfl_lst.append(key)
# lst = {}
# for key in nfl_lst:
#     lst[key[5:]] = json_dict[key]
#     del json_dict[key]
# if lst:
#     json_dict['nfl_draft'] = lst

#suppdraft
# supp_lst = []
# for key in json_dict.keys():
#     if 'suppdraft' in key:
#         supp_lst.append(key)
# lst = {}
# for key in supp_lst:
#     lst[key[9:]] = json_dict[key]
#     del json_dict[key]
# if lst:
#     json_dict['supplemental_draft'] = lst

# #afldraft
# afl_lst = []
# for key in json_dict.keys():
#     if 'afldraft' in key:
#         afl_lst.append(key)
# lst = {}
# for key in afl_lst:
#     lst[key[8:]] = json_dict[key]
#     del json_dict[key]
# if lst:
#     json_dict['afl_draft'] = lst
    
#statleague
# stats = []
# for key in json_dict.keys():
#     for i in range(1,18):
#         if (f'statlabel{i}' in json_dict.keys() and f'statvalue{i}' in json_dict.keys()):
#             stats.append(i)
# stats = list(set(stats))
# lst = {}
# for num in stats:
#     lst[json_dict[f'statlabel{num}']] = json_dict[f'statvalue{num}']
#     del json_dict[f'statlabel{num}']
#     del json_dict[f'statvalue{num}']
# if lst:
#     try:
#         json_dict[json_dict['statleague']] = lst
#         del json_dict['statleague']
#     except:
#         json_dict['stats'] = lst
    
# #nationalteams
nt_lst = []    
for i in range(1,18):
    if f'nationalyears{i}' in json_dict.keys():
        nt_lst.append(i)

lst = {}
for num in nt_lst:
    try:
        lst[json_dict[f'nationalteam{num}']] = json_dict[f'nationalyears{num}']
    except:
        pass
            
    try:
        del json_dict[f'nationalteam{num}']
    except:
        pass
    
    try:
        del json_dict[f'nationalyears{num}']
    except:
        pass
    
if lst:
    json_dict['nationalteams'] = lst
    
# Club Structuring
club_lst = []    
for i in range(1,18):
    if f'years{i}' in json_dict.keys():
        club_lst.append(i)

lst = {}
for num in club_lst:
    try:
        lst[json_dict[f'team{num}']] = json_dict[f'years{num}']
    except:
        pass
            
    try:
        del json_dict[f'team{num}']
    except:
        pass
    
    try:
        del json_dict[f'years{num}']
    except:
        pass
    
if lst:
    json_dict['clubs'] = lst
    
# Club Structuring
award_lst = []    
for i in range(1,18):
    if f'year{i}' in json_dict.keys():
        award_lst.append(i)

lst = {}
for num in award_lst:
    try:
        lst[json_dict[f'award{num}']] = json_dict[f'year{num}']
    except:
        pass
            
    try:
        del json_dict[f'award{num}']
    except:
        pass
    
    try:
        del json_dict[f'year{num}']
    except:
        pass
    
if lst:
    json_dict['awards'] = lst

    
try:
    if birth_date:
        json_dict['birth_date'] = birth_date[:-1]
except:
    pass    

try:
    if global_lst:
        json_dict['medaltemplates'] = global_lst
    else:
        json_dict['medaltemplates'] = raw_dict['medaltemplates']
except:
    pass

try:
    if championship_years:
        json_dict['WorldOpenresult'] = championship_years
except:
    pass
# # try:
#     fide_id = int(json_dict['FideID'].strip())
#     url = f"http://ratings.fide.com/card.phtml?event={fide_id}"
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     try:
#         std_rating = re.findall('\d\d\d\d', soup.find('div',{'class':'profile-top-rating-data profile-top-rating-data_gray'}).text)[0]
#         json_dict['std_rating'] = std_rating
#     except:
#         pass
#     try:
#         rapid_rating = re.findall('\d\d\d\d', soup.find('div',{'class':'profile-top-rating-data profile-top-rating-data_red'}).text)[0]
#         json_dict['rapid_rating'] = rapid_rating
#     except:
#         pass
#     try:
#         blitz_rating = re.findall('\d\d\d\d', soup.find('div',{'class':'profile-top-rating-data profile-top-rating-data_blue'}).text)[0]
#         json_dict['blitz_rating'] = blitz_rating
#     except:
#         pass
#     try:
#         ranking = re.findall('\d', soup.find('div',{'class':'profile-top-info__block__row__data'}).text)[0]
#         json_dict['current_ranking'] = ranking
#     except:
#         try:
#             ranking = re.findall('\d\d', soup.find('div',{'class':'profile-top-info__block__row__data'}).text)[0]
#             json_dict['current_ranking'] = ranking
#         except:
#             pass
# except:
#     pass

json_dict

{'station': 'International Space Station',
 'COSPAR_ID': '1998-067A',
 'SATCAT': '25544',
 'sign': "''Alpha'', ''Station''",
 'crew': 'Fully crewed: 7 ',
 'launch': '1998',
 'launch_pad': ['nowrap', 'Kennedy, LC-39 and CCSFS, SLC-40'],
 'mass': '450000',
 'title': 'ISS: International Space Station ',
 'length': '358 (Overall length), 310 (truss length)  last=Garcia ',
 'width': '239 (solar array length)',
 'volume': '35491',
 'pressure': '101.3 79% nitrogen, 21% oxygen',
 'perigee': '413 AMSL last=Peat ',
 'apogee': '422 AMSL',
 'inclination': '51.64°',
 'speed': '7.66',
 'period': '92.9 minutes',
 'orbits_day': '15.49',
 'in_orbit': '20 November 1998 06:40 (TODAY)',
 'occupied': '2 November 2000 09:21 (TODAY)',
 'orbits': '133,312  2022',
 'decay': '2 km/month',
 'orbit_epoch': '12 October 2022 14:25:10  last=Holman ',
 'apsis': 'gee',
 'as_of': '22 December 2022 (unless noted otherwise)'}

In [19]:
import mwparserfromhell

wikicode = "{{Birth date and age|df=y|1971|11|21}}"
parsed_code = mwparserfromhell.parse(wikicode)
date_template = parsed_code.filter_templates()[0]
birth_date = ''
try:
    birth_date = birth_date + date_template.get(1).value.strip() + '-' 
except:
    pass
try:
    birth_date = birth_date + date_template.get(2).value.strip() + '-' 
except:
    pass
try:
    birth_date = birth_date + date_template.get(3).value.strip() + '-'
except:
    pass
try:
    birth_date = birth_date + date_template.get(4).value.strip()
except:
    pass
print(birth_date)

1971-11-21-


In [50]:
infobox

'{{short description|Swiss alpine skier}}\n{{Use dmy dates|date=September 2022}}\n{{Infobox alpine ski racer\n| name          = Maria Walliser\n| image         = Maria Walliser.png\n|  image_size    = \n| caption       = \n| disciplines   = Speed events, Giant slalom\n| club          = SC Libingen\n| birth_date    = {{birth date and age|27 May 1963|df=yes}}\n| birth_place   = [[Mosnang]], [[Switzerland]]\n| death_date    = \n| death_place   = \n| height        = 1.68 m\n| wcdebut       = 1983\n| retired       = 1990\n| website       = \n| olympicteams  = 2\n| olympic medals = 3\n| olympicgolds  = \n| worldsteams   = 4\n| worlds medals  = 4\n| worldsgolds   = 3\n| wcseasons     = 8\n| wcwins        = 25\n| wcpodiums     = 72\n| wcoveralls    = 2\n| wctitles      = 5\n|show- medals   = yes\n|med altemplates= \n{{Medal|Sport|Women\'s [[alpine skiing]]}}\n{{Medal|Country|{{SUI}}}} \n{{MedalCount|total=yes|type=World Cup race podiums\n| Slalom       | 0 | 0 | 1\n| Giant slalom | 6 | 5 | 6\n

In [68]:
raw_dict

{'name': ' Hanni Wenzel',
 'image': ' Hanni Wenzel.png',
 'image_size': ' ',
 'caption': '  ',
 'disciplines': ' [[Giant slalom|Giant Slalom]], [[Slalom skiing|Slalom]],<br />[[Alpine skiing combined|Combined]], [[Downhill (ski competition)|Downhill]],<br>[[Super-G|Super G]]',
 'club': ' ',
 'skis': ' ',
 'boots': ' ',
 'bindings': ' ',
 'sponsor': ' ',
 'birth_date': ' {{Birth date and age|1956|12|14|df=y}}',
 'birth_place': ' [[Straubing]], [[Bavaria]],<br>[[West Germany]]',
 'death_date': ' ',
 'death_place': ' ',
 'height': ' 1.65 m',
 'weight': ' ',
 'wcdebut': ' 1 March 1972 (age 15)',
 'retired': ' March 1984  (age 27)',
 'website': ' [http://www.wwp-group.com/en wwp-group.com]',
 'olympicteams': ' 2 – ([[Alpine skiing at the 1976 Winter Olympics|1976]], [[Alpine skiing at the 1984 Winter Olympics|1980]])',
 'olympicmedals': ' 4'}

In [9]:
# timeline_keys = ['birth_date', 'medaltemplates', 'yearsactive', 'rating', 'peakrating', 'ranking', 'peakranking', 'title',
#                  'worldchampion', 'womensworldchampion ']
# final_timeline_keys = ['medaltemplates', 'yearsactive', 'rating', 'peakrating', 'ranking', 'peakranking', 'title',
#                  'worldchampion', 'womensworldchampion ']
timeline_keys = ['birth_date', 'medaltemplates', 'years', 'clubs', 'nationalteams', 'nationalyears']

final_timeline_keys = ['medaltemplates', 'clubs', 'years', 'nationalteams', 'nationalyears']

timeline = {}
for key in timeline_keys:
    try:
        timeline[key] = raw_dict[key]
    except:
        pass
# medal_templates

In [10]:
final_timeline = {}
# try:
#     final_timeline['date_of_highest_ranking'] = timeline['date_of_highest_ranking']
# except:
#     pass
# try:
#     final_timeline['date_of_current_ranking'] = timeline['date_of_current_ranking']
# except:
#     pass
# try:
#     final_timeline['turnedpro'] = [timeline['turnedpro']] 
# except:
#     pass

# try:
#     lst = [re.findall("\d\d\d\d", ele) for ele in [json_dict["years_active"]] if re.findall("\d\d\d\d", ele)]
#     lst = [item for sublist in lst for item in sublist]
#     final_timeline['years_active'] = list(set(lst))
# except:
#     pass

# try:
#     lst = [re.findall("\d\d\d\d", ele) for ele in json_dict["medaltemplates"] if re.findall("\d\d\d\d", ele)]
#     lst = [item for sublist in lst for item in sublist]
#     final_timeline['medaltemplates'] = list(set(lst))
# except:
#     pass
# for key in raw_dict.keys():
try:
    lst = [re.findall("\d\d\d\d", ele) for ele in raw_dict[key] if re.findall("\d\d\d\d", ele)]
    lst = [item for sublist in lst for item in sublist]
    final_timeline[key] = list(set(lst))
    if (final_timeline[key] == []):
        lst = re.findall("\d\d\d\d", raw_dict[key])
        final_timeline[key] = list(set(lst))
except:
    pass
# for key in raw_dict.keys():
#     if "year" in key:
#         try:
#             lst = [re.findall("\d\d\d\d", ele) for ele in raw_dict[key] if re.findall("\d\d\d\d", ele)]
#             lst = [item for sublist in lst for item in sublist]
#             final_timeline[key] = list(set(lst))
#             if (final_timeline[key] == []):
#                 lst = re.findall("\d\d\d\d", raw_dict[key])
#                 final_timeline[key] = list(set(lst))
#         except:
#             pass
# try:
#     lst = [re.findall("\d\d\d\d", ele) for ele in json_dict["WorldOpenResult"] if re.findall("\d\d\d\d", ele)]
#     lst = [item for sublist in lst for item in sublist]
#     final_timeline['WorldOpenResult'] = list(set(lst))
# except:
#     pass
final_timeline

{}

In [11]:
timelines = re.findall("\d\d\d\d", infobox2)
final_timelines = []
for ele in timelines:
    if (int(ele) <= 2023) and (int(ele) >= 2000):
        final_timelines.append(ele)
        
list(set(final_timelines))

['2022',
 '2010',
 '2018',
 '2021',
 '2012',
 '2014',
 '2020',
 '2016',
 '2006',
 '2019']

In [14]:
links = dict()
links[page_title] = []
for ele in final_timelines:
    datei = dateparser.parse(ele)
    start = datei.replace(month=1, day = 1).strftime("%Y%m%d") + '000000'
    end = datei.replace(month=12, day = 31).strftime("%Y%m%d") + '000000'
    links[page_title]  = links[page_title] + get_edit_history(page_title, start, end)

NameError: name 'get_edit_history' is not defined

In [ ]:
datei

In [13]:
import dateparser
links = dict()
links[page_title] = []
for key in final_timeline:
    try:
        date = dateparser.parse(final_timeline[key])
        try:
            if date.month == 1:
                start = date.replace(month=12, year = date.year-1).strftime("%Y%m%d") + '000000'
            else:
                start  = date.replace(month=date.month - 1).strftime("%Y%m%d") + '000000'
        except:
            start  = date.replace(month=1).strftime("%Y%m%d") + '000000'
        try:
            if date.month == 12:
                end = date.replace(month=1, year = date.year+1).strftime("%Y%m%d") + '000000'
            else:
                end = date.replace(month=date.month + 1).strftime("%Y%m%d") + '000000'
        except:
            end = date.replace(month=12).strftime("%Y%m%d") + '000000'
    except:
        try:
            dates = [dateparser.parse(ele) for ele in final_timeline[key]]
            start = []
            end = []
            for date in dates:
                starti = date.replace(month=1).strftime("%Y%m%d") + '000000'
                endi = date.replace(month=12).strftime("%Y%m%d") + '000000'
                start.append(starti)
                end.append(endi)
        except:
            pass
#     print("Key: ", key)
    print("Start: ", start)
    print("End: ", end)
#     try:
#         links[page_title]  = links[page_title] + get_edit_history(page_title, start, end)
#     except:
#         try:
#             for i in range(len(start)):
#                 links[page_title]  = links[page_title] + get_edit_history(page_title, start[i], end[i])
#         except:
#             pass

# links[page_title] = [d for i, d in enumerate(links[page_title]) if d not in links[page_title][:i]]

In [ ]:
clean(medal_templates[3].get(2).strip())

In [ ]:
text = raw_dict['medaltemplates']
medals = {}
wikicode = parse(text)
medal_templates = wikicode.filter_templates()

for template in medal_templates:
    if template.name.matches("MedalSport"):
        medals["sport"] = clean_square(template.get(1).value.strip())
    elif template.name.matches("MedalCountry"):
        medals["country"] = clean_square(template.get(1).value.strip())
    elif template.name.matches("MedalCompetition"):
        competition = clean_square(template.get(1).value.strip())
        if competition not in medals:
            medals[competition] = {}
    elif template.name.matches("MedalGold"):
        competition = clean_square(template.get(1).value.strip())
        year_event = clean_square(template.get(2).value.strip())
        event = clean_square(template.get(3).value.strip())
        if event not in medals[competition]:
            medals[competition][event] = {}
        medals[competition][event]["gold"] = year_event
    elif template.name.matches("MedalSilver"):
        competition = clean_square(template.get(1).value.strip())
        year_event = clean_square(template.get(2).value.strip())
        event = clean_square(template.get(3).value.strip())
        if event not in medals[competition]:
            medals[competition][event] = {}
        medals[competition][event]["silver"] = year_event
    elif template.name.matches("MedalBronze"):
        competition = clean_square(template.get(1).value.strip())
        year_event = clean_square(template.get(2).value.strip())
        event = clean_square(template.get(3).value.strip())
        if event not in medals[competition]:
            medals[competition][event] = {}
        medals[competition][event]["bronze"] = year_event

In [ ]:
clean(medal_templates[3].get(6).value.strip())

In [ ]:
medal_templates[0]

In [ ]:
text = raw_dict['medaltemplates']
wikicode = parse(value)
medal_templates = wikicode.filter_templates()

for template in medal_templates:
    if template.name.matches("MedalSport"):
        global_lst.append("Sport: "+clean(template.get(1).value.strip()))
    elif template.name.matches("MedalCountry"):
        global_lst.append("Country: "+clean_square(template.get(1).value.strip()))
    elif template.name.matches("MedalCompetition"):
        competition = clean_square(template.get(1).value.strip())
        global_lst.append("Competition: "+competition)
    elif template.name.matches("MedalGold"):
        competition = clean_square(template.get(1).value.strip())
        year_event = clean_square(template.get(2).value.strip())
        event = clean_square(template.get(3).value.strip())
        global_lst.append('Gold Medal: '+competition+' '+year_event + ' ' + event)
    elif template.name.matches("MedalSilver"):
        competition = clean_square(template.get(1).value.strip())
        year_event = clean_square(template.get(2).value.strip())
        event = clean_square(template.get(3).value.strip())
        global_lst.append('Silver Medal: '+competition+' '+year_event + ' ' + event)
    elif template.name.matches("MedalBronze"):
        competition = clean_square(template.get(1).value.strip())
        year_event = clean_square(template.get(2).value.strip())
        event = clean_square(template.get(3).value.strip())
        global_lst.append('Bronze Medal: '+competition+' '+year_event + ' ' + event)

In [ ]:
text = raw_dict['birth_date']
result = remove_refs(text)
print(result)
result = remove_pixel_info(result)
print(result)
result = remove_html(result) 
print(result)
result = replace_special(result)
print(result)
lst = extract_text_between_braces(result)
lst_alt = []
for ele in lst:
    if '|' in ele:
        result = result.replace(ele, ele.split('|')[1])
print(result)
lst = extract_text_between_curly_braces(result)
lst_alt = []
for ele in lst:
    if ('|' in ele) and ('hlist' not in ele) and ('birth' not in ele):
        result = result.replace(ele, ele.split('|')[1])
result = re.sub('[\[\]]+','',result)
print(result)

In [ ]:
raw_dict_inter

In [ ]:
import requests

def get_edit_history(page_title, start_date, end_date):
    # Initialize an empty list to store the revisions
    revisions = []
    
    # Set the base URL for the API request
    api_url = "https://en.wikipedia.org/w/api.php"
    
    # Set the parameters for the API request
    params = {
        "action": "query",
        "titles": page_title,
        "rvstart":start_date,
        "rvend":end_date,
        "format": "json",
        "prop": "revisions",
        "rvlimit": "500",
        "rvdir": "newer"
    }
    
    # Set a flag to indicate whether there are more revisions to retrieve
    more_revisions = True
    
    while more_revisions:
        # Make the API request
        response = requests.get(api_url, params=params)
        
        # Get the data from the response
        data = response.json()
        
        # Extract the revisions from the data
        page_id = list(data["query"]["pages"].keys())[0]
        page_revisions = data["query"]["pages"][page_id]["revisions"]
        
        # Add the revisions to the list
        revisions.extend(page_revisions)
        
        # Check if there are more revisions to retrieve
        if "continue" in data:
            # Update the rvcontinue parameter for the next request
            params["rvcontinue"] = data["continue"]["rvcontinue"]
        else:
            # Set the flag to False to stop the loop
            more_revisions = False
    
    # Return the list of revisions
    return revisions
# get_edit_history("Natalie_Grainger", start[1], end[1])

In [50]:
import mwparserfromhell

wikicode = """
'''[[Grand Tour (cycling)|Grand Tours]]''' <br/> :'''[[Tour de France]]''' :: 1 individual stage ([[2003 Tour de France|2003]]) :'''[[Giro d'Italia]]''' :: 1 individual stage ([[2002 Giro d'Italia|2002]]) :'''[[Vuelta a España]]''' :: 1 individual stage ([[2004 Vuelta a España|2004]]) '''[[Race stage|Stage races]]''' :[[Tour de Romandie]] (2003, 2004) :[[Dauphiné Libéré]] (2000) '''[[Classic cycle races|Single-day races and Classics]]''' :{{nowrap|[[United States National Road Race Championships|National Road Race Championships]] (2008)}} :[[Liège–Bastogne–Liège]] (2003)
"""

wikicode = mwparserfromhell.parse(wikicode)

# Extracting the contents of each line
for line in wikicode.filter_text():
    print(line)




Grand Tour (cycling)
Grand Tours
 
br
 :
Tour de France
 :: 1 individual stage (
2003 Tour de France
2003
) :
Giro d'Italia
 :: 1 individual stage (
2002 Giro d'Italia
2002
) :
Vuelta a España
 :: 1 individual stage (
2004 Vuelta a España
2004
) 
Race stage
Stage races
 :
Tour de Romandie
 (2003, 2004) :
Dauphiné Libéré
 (2000) 
Classic cycle races
Single-day races and Classics
 :
nowrap
United States National Road Race Championships
National Road Race Championships
 (2008)
 :
Liège–Bastogne–Liège
 (2003)



In [123]:
pd.read_table('http://bit.ly/uforeports')

,"City,Colors Reported,Shape Reported,State,Time"
0,"Ithaca,,TRIANGLE,NY,6/1/1930 22:00"
1,"Willingboro,,OTHER,NJ,6/30/1930 20:00"
2,"Holyoke,,OVAL,CO,2/15/1931 14:00"
3,"Abilene,,DISK,KS,6/1/1931 13:00"
4,"New York Worlds Fair,,LIGHT,NY,4/18/1933 19:00"
...,...
18236,"Grant Park,,TRIANGLE,IL,12/31/2000 23:00"
18237,"Spirit Lake,,DISK,IA,12/31/2000 23:00"
18238,"Eagle River,,,WI,12/31/2000 23:45"
18239,"Eagle River,RED,LIGHT,WI,12/31/2000 23:45"
